In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import h5py
from qutip import *
from estimation import *
from data_processing import *
from scipy.io import savemat
from targets import *
import exp_params

USING W-FUNCTION POINTS


### Check yourself: print function, dimension, and number of displacements

In [3]:
print(f"{exp_params.which_function}\nD={exp_params.D}\nnD={exp_params.nD}")

W
D=6
nD=35


### State list

In [4]:
state_list = exp_params.cat_state_list
print(state_list)

['cat-eve-1' 'cat-nmp-1' 'cat-nop-1' 'cat-odd-1']


### Experimental data directory

In [5]:
exp_data_directory = r"data\fidelity_v_t2\W"
all_files = np.array(os.listdir(exp_data_directory))
print(all_files)

['032612_YABBA V3_BadT2_cats_Wigner_grape_cat-eve-1_point0_No0_ampx0.1times0.0_QubitFre-119.97965780980907_.h5'
 '032656_YABBA V3_BadT2_cats_Wigner_grape_cat-eve-1_point0_No1_ampx0.1times0.075_QubitFre-119.99534614598208_.h5'
 '032740_YABBA V3_BadT2_cats_Wigner_grape_cat-eve-1_point0_No2_ampx0.1times0.15_QubitFre-120.06052346537744_.h5'
 '032824_YABBA V3_BadT2_cats_Wigner_grape_cat-eve-1_point0_No3_ampx0.1times0.25_QubitFre-120.23561673827437_.h5'
 '032908_YABBA V3_BadT2_cats_Wigner_grape_cat-eve-1_point0_No4_ampx0.1times0.3_QubitFre-120.3491472072318_.h5'
 '032952_YABBA V3_BadT2_cats_Wigner_grape_cat-eve-1_point1_No0_ampx0.1times0.0_QubitFre-119.97965780980907_.h5'
 '033036_YABBA V3_BadT2_cats_Wigner_grape_cat-eve-1_point1_No1_ampx0.1times0.075_QubitFre-119.99534614598208_.h5'
 '033120_YABBA V3_BadT2_cats_Wigner_grape_cat-eve-1_point1_No2_ampx0.1times0.15_QubitFre-120.06052346537744_.h5'
 '033204_YABBA V3_BadT2_cats_Wigner_grape_cat-eve-1_point1_No3_ampx0.1times0.25_QubitFre-120.23561

### Directory for target states generated from grape (we don't use ideal when we're processing experimental data)

In [6]:
# GRAPE generated target states
target_states_directory = r"target_states"


### Gather linear inversion variables used to generate Least-Squares estimator, $\rho_{LS}$

In [7]:
map_variables = np.load(
    f"map_variables\map_variables_D={exp_params.D}_nD={exp_params.nD}_W.npz"
)


W = map_variables["W"]

beta = map_variables["beta"]

### Top level iteration through the T2s. Then, iterate through the states. For each state: 1) collect the observable outcomes, 2) perform linear inversion with the map to get $\rho_{LS}$, 3) perform both MLE and Bayesian, and 4) record in numpy arrays the fidelities, standard deviations, and $\rho_{BME}$ for each. This saves both the normal Wigner and corrected Wigner information.

In [8]:
F_MLE_corr = np.zeros(len(state_list), dtype=float)
F_MLE_norm = np.zeros(len(state_list), dtype=float)


F_Bayes_BME_corr = np.zeros(len(state_list), dtype=float)
F_Bayes_BME_norm = np.zeros(len(state_list), dtype=float)
final_rho_BME_corr = np.zeros(
    (len(state_list), exp_params.D, exp_params.D), dtype="complex_"
)
final_rho_BME_norm = np.zeros(
    (len(state_list), exp_params.D, exp_params.D), dtype="complex_"
)
final_rho_MLE_corr = np.zeros(
    (len(state_list), exp_params.D, exp_params.D), dtype="complex_"
)
final_rho_MLE_norm = np.zeros(
    (len(state_list), exp_params.D, exp_params.D), dtype="complex_"
)

In [9]:
RR_No_list = [0, 1, 2, 3, 4]

for rrno in RR_No_list:
    print(f"T2: {rrno}\n\n\n")
    for j, state_name in enumerate(state_list):  # State List
        data_corr = np.zeros([exp_params.nD])  # Initialize data vector


        data_norm = np.zeros([exp_params.nD])  # Initialize data vector


        for point in range(1, exp_params.nD + 1):  # Point1 to Point35
            ending = (
                "_grape_" + str(state_name) + "_point" + str(point - 1) + f"_No{rrno}"
            )
            matching = [
                filename
                for filename in all_files
                if (filename.split("_ampx")[0]).endswith(str(ending))
            ]
        
            filepath = exp_data_directory + "/" + matching[0]
            file = h5py.File(filepath, "r")


            # selected_data, selected_data_m = select2_flat(filepath, postSel)


            signal, signal_m = post_selection_W(filepath)


            # signal_m = np.average(selected_data_m)


            if state_name in exp_params.pes_after_grape[:, 0]:
                pge = exp_params.pes_after_grape[
                    (exp_params.pes_after_grape[:, 0] == state_name), 1
                ].astype(float)


            else:
                pge = exp_params.pe_after_grape_avg


            aux = (signal - pge) / (1 - 2 * pge)


            aux_m = (signal_m - pge) / (1 - 2 * pge)


            data_corr[point - 1] = aux - aux_m


            data_norm[point - 1] = 2 * aux - 1


        rho_est_corr, qRho_est_corr = get_LS_and_MLE_rho_est(
            data_corr, W, beta, exp_params.D, exp_params.nD
        )


        rho_est_norm, qRho_est_norm = get_LS_and_MLE_rho_est(
            data_norm, W, beta, exp_params.D, exp_params.nD
        )


        rho_tar_qc = Y_target(state_name, target_states_directory, qdim=3, cdim=30)


        rho_tar_c = Qobj(rho_tar_qc.ptrace(1)[0 : exp_params.D, 0 : exp_params.D])


        rho_tar_c = rho_tar_c / rho_tar_c.tr()  # normalise it, .unit()


        Fmean_bayes_corr, Fstd_bayes_corr, rho_BME_corr = bayesian_rho_est(
            numSamp=2**10,
            N=exp_params.N_exp,
            rho_tar=rho_tar_c,

            rhoLS=rho_est_corr.full(),
        )


        Fmean_bayes_norm, Fstd_bayes_norm, rho_BME_norm = bayesian_rho_est(
            numSamp=2**10,
            N=exp_params.N_exp,
            rho_tar=rho_tar_c,

            rhoLS=rho_est_norm.full(),
        )


        F_Bayes_BME_corr[j] = fidelity(rho_tar_c, Qobj(rho_BME_corr)) ** 2


        F_Bayes_BME_norm[j] = fidelity(rho_tar_c, Qobj(rho_BME_norm)) ** 2


        F_MLE_corr[j] = fidelity(rho_tar_c, qRho_est_corr) ** 2


        F_MLE_norm[j] = fidelity(rho_tar_c, qRho_est_norm) ** 2


        final_rho_BME_corr[j, :, :] = rho_BME_corr


        final_rho_BME_norm[j, :, :] = rho_BME_norm


        final_rho_MLE_corr[j, :, :] = qRho_est_corr


        final_rho_MLE_norm[j, :, :] = qRho_est_norm


        print(f"state {j}: {state_name}, corr: {F_Bayes_BME_corr[j]}")


        print(f"state {j}: {state_name}, norm: {F_Bayes_BME_norm[j]}\n")
    np.savez(
        f"results_t2\exp\W\Bayes_F_rrno={rrno}_BME_MLE_rho_corr_norm_W.npz",
        F_Bayes_BME_corr=F_Bayes_BME_corr,
        F_Bayes_BME_norm=F_Bayes_BME_norm,
        F_MLE_corr=F_MLE_corr,
        F_MLE_norm=F_MLE_norm,
        final_rho_BME_corr=final_rho_BME_corr,
        final_rho_BME_norm=final_rho_BME_norm,
        final_rho_MLE_corr=final_rho_MLE_corr,
        final_rho_MLE_norm=final_rho_MLE_norm,
    )

T2: 0





C:\Users\clara\AppData\Local\Temp\ipykernel_3772\89378867.py:51: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  data_corr[point - 1] = aux - aux_m
C:\Users\clara\AppData\Local\Temp\ipykernel_3772\89378867.py:54: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  data_norm[point - 1] = 2 * aux - 1


state 0: cat-eve-1, corr: 0.9686344870702053
state 0: cat-eve-1, norm: 0.6571384823258746

state 1: cat-nmp-1, corr: 0.9490482296388673
state 1: cat-nmp-1, norm: 0.5205901798506939

state 2: cat-nop-1, corr: 0.9609003076940056
state 2: cat-nop-1, norm: 0.5870902062546108

state 3: cat-odd-1, corr: 0.9391573263466831
state 3: cat-odd-1, norm: 0.4542644214528204

T2: 1



state 0: cat-eve-1, corr: 0.9362020726383777
state 0: cat-eve-1, norm: 0.6235644830669385

state 1: cat-nmp-1, corr: 0.9318257955483342
state 1: cat-nmp-1, norm: 0.4896225509531006

state 2: cat-nop-1, corr: 0.9185633649961624
state 2: cat-nop-1, norm: 0.5529869622188125

state 3: cat-odd-1, corr: 0.9313196915916486
state 3: cat-odd-1, norm: 0.4227851384897934

T2: 2



state 0: cat-eve-1, corr: 0.896378520654185
state 0: cat-eve-1, norm: 0.5905409271848301

state 1: cat-nmp-1, corr: 0.8744326556767892
state 1: cat-nmp-1, norm: 0.44024049811745297

state 2: cat-nop-1, corr: 0.9198838658053237
state 2: cat-nop-1, norm: 0